# Deep Learning Classifier

In [13]:
import pandas as pd

train = pd.read_csv('../train.csv')
test = pd.read_csv('../test.csv')

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input
import optuna

2024-05-08 01:54:22.100820: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


ModuleNotFoundError: No module named 'optuna'

In [15]:
# Escalar características
scaler = MinMaxScaler()
X_train = scaler.fit_transform(train.drop('nobeyesdad', axis=1))
X_test = scaler.transform(test.drop('nobeyesdad', axis=1))

# Codificar la variable objetivo
y_train = to_categorical(train['nobeyesdad'].astype('category').cat.codes)
y_test = to_categorical(test['nobeyesdad'].astype('category').cat.codes)

In [28]:
def create_model(trial):
    n_layers = trial.suggest_int("n_layers", 1, 20)
    model = Sequential()
    model.add(Input(trial.suggest_int("n_units_l0", 5, 200), activation=trial.suggest_categorical("activation_l0", ["relu", "tanh", "sigmoid"]), input_shape=(X_train.shape[1],)))
    model.add(Dropout(trial.suggest_float("dropout_l0", 0.1, 0.5)))
    for i in range(n_layers):
        num_units = trial.suggest_int(f"n_units_l{i+1}", 5, 200)
        activation = trial.suggest_categorical(f"activation_l{i+1}", ["relu", "tanh", "sigmoid"])
        dropout = trial.suggest_float(f"dropout_l{i+1}", 0.1, 0.5)
        model.add(Dense(num_units, activation=activation))
        model.add(Dropout(dropout))
    model.add(Dense(y_train.shape[1], activation="softmax"))
    return model

def objective(trial):
    model = create_model(trial)
    optimizer_options = trial.suggest_categorical("optimizer", ["adam", "sgd", "rmsprop"])
    model.compile(optimizer=optimizer_options,
                  loss="categorical_crossentropy",
                  metrics=["accuracy"])
    
    model.fit(X_train, y_train, verbose=0, epochs=50, batch_size=32, validation_split=0.2)
    loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
    return accuracy



In [26]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

print("Best trial:")
trial = study.best_trial
print(f"  Value: {trial.value}")
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")
